# Psycopg – wykonywanie SQL w Pythonie

Biblioteka służąca do wykonywania kodu SQL na bazie danych z poziomu Pythona. Jest przystosowana wyłącznie do pracy z systemem PostgreSQL. Inne systemy mają swoje odpowiedniki, np.:

- [MySQL](https://pypi.org/project/mysql-connector-python/)
- [Oracle](https://pypi.org/project/oracledb/)


Najnowsza wersja ma numer 3 ([PyPI](https://pypi.org/project/psycopg/)) chociaż cały czas popularna jest wersja 2 ([PyPI](https://pypi.org/project/psycopg2/)).

In [9]:
import psycopg

In [10]:
conn = psycopg.connect(dbname="postgres", user="postgres", password="postgres", host="localhost")

cursor = conn.cursor()

## Zapytania SQL

### CREATE

In [12]:
create_query = """CREATE TABLE tasks(
    id SERIAL,
    description TEXT,
    priority SMALLINT,
    is_completed BOOLEAN
)
"""

In [13]:
cursor.execute(create_query)
conn.commit()

In [14]:
create_query = """CREATE TABLE users(
    id SERIAL,
    username VARCHAR(16),
    password VARCHAR(36),
    is_admin BOOLEAN
)
"""

In [15]:
cursor.execute(create_query)
conn.commit()

### INSERT

In [16]:
insert_query = "INSERT INTO tasks(description, priority, is_completed) VALUES('Opis zadania', 2, true)"

In [17]:
cursor.execute(insert_query)
conn.commit()

#### Klauzula `RETURNING`

In [19]:
insert_query = """INSERT INTO tasks (description, priority, is_completed)
                  VALUES ('Zrobić obiad', 3, false)  RETURNING *"""

In [20]:
cursor.execute(insert_query)

record = cursor.fetchone()
record

(2, 'Zrobić obiad', 3, False)

In [21]:
record[0]

2

In [22]:
conn.commit()

### SELECT

In [24]:
select_query = "SELECT * FROM tasks"

In [26]:
cursor.execute(select_query)

results = cursor.fetchall()

In [27]:
results

[(1, 'Opis zadania', 2, True), (2, 'Zrobić obiad', 3, False)]

**Wiersze jako słowniki**

In [28]:
cursor = conn.cursor(row_factory=psycopg.rows.dict_row)

In [29]:
cursor.execute(select_query)
results = cursor.fetchall()
results

[{'id': 1, 'description': 'Opis zadania', 'priority': 2, 'is_completed': True},
 {'id': 2,
  'description': 'Zrobić obiad',
  'priority': 3,
  'is_completed': False}]

In [32]:
results[0]

{'id': 1, 'description': 'Opis zadania', 'priority': 2, 'is_completed': True}

In [33]:
cursor = conn.cursor()

### UPDATE

In [44]:
update_query = """UPDATE tasks SET description='Nauczyć się SQLa'
                  WHERE id=1 RETURNING *"""

In [45]:
cursor.execute(update_query)
record = cursor.fetchone()
record

(1, 'Nauczyć się SQLa', 2, True)

In [46]:
conn.commit()

### DELETE

In [47]:
delete_query = "DELETE FROM tasks WHERE id=2;"
cursor.execute(delete_query)
conn.commit()

### DROP

In [48]:
drop_query = "DROP TABLE users;"

cursor.execute(drop_query)
conn.commit()

## Template'y zapytań i zapobieganie SQL injection 

In [49]:
id_number = '1'

query = f"UPDATE tasks SET priority = 3 WHERE id = {id_number}"
query

'UPDATE tasks SET priority = 3 WHERE id = 1'

In [50]:
cursor.execute(query)
conn.commit()

---

In [51]:
# przygotowanie tabeli do usunięcia

create_query = "CREATE TABLE test();"
cursor.execute(create_query)
conn.commit()

SQL injection:

In [52]:
id_number = '1;  DROP TABLE test;'


query = f"UPDATE tasks SET priority = 4 WHERE id = {id_number}"
query

'UPDATE tasks SET priority = 4 WHERE id = 1;  DROP TABLE test;'

In [53]:
cursor.execute(query)
conn.commit()

---

In [54]:
id_number = '1'

query = "UPDATE tasks SET priority = 5 WHERE id = %s"

cursor.execute(query, (id_number,))
conn.commit()

Gdybyśmy chcieli zobaczyć podgląd takiego zapytania, możemy zrobić to korzystając z następującego feature'a pythonowych stringów:

In [56]:
query % (id_number,)

'UPDATE tasks SET priority = 5 WHERE id = 1'

---

In [57]:
create_query = "CREATE TABLE test();"
cursor.execute(create_query)
conn.commit()

In [58]:
id_number = '1; DROP TABLE test;'

query = "UPDATE tasks SET priority = 6 WHERE id = %s"

cursor.execute(query, (id_number,))

InvalidTextRepresentation: invalid input syntax for type integer: "1; DROP TABLE test;"
CONTEXT:  unnamed portal parameter $1 = '...'

In [60]:
query % (id_number,)

'UPDATE tasks SET priority = 6 WHERE id = 1; DROP TABLE test;'

In [61]:
conn.rollback()

> **ZADANIA**

In [62]:
cursor.close()
conn.close()